In [15]:
import pandas as pd
import numpy as np

df = pd.read_csv('data/student_depression_dataset.csv', index_col=0)
df.shape

(27901, 17)

In [16]:
df.head()
df.shape
#print(df.isnull().sum())

(27901, 17)

In [17]:
#Drop rows where the following columns are Others
# print(df['Dietary Habits'].unique())
print(df['Sleep Duration'].unique())


#print(df['City'].unique())


# print(df['Work Pressure'].unique())
# print((df['Work Pressure'] == 2).sum())

# print(df['Job Satisfaction'].unique())
# print((df['Job Satisfaction'] == 1).sum())

#Drop columns where Profession is not Student
#print(df['Profession'].value_counts())

# print(df['Academic Pressure'].value_counts())
#print(df['Degree'].value_counts())
print((df['Degree'].value_counts()).count())
print((df['City'].value_counts()).count())

["'5-6 hours'" "'Less than 5 hours'" "'7-8 hours'" "'More than 8 hours'"
 'Others']
28
52


In [18]:
print((df['Dietary Habits'] == 'Others').sum())
print((df['Sleep Duration'] == 'Others').sum())
print(((df['Dietary Habits'] == 'Others') & (df['Sleep Duration'] == 'Others')).sum())

12
18
0


In [19]:
filtered_df = df.copy()
print(filtered_df.shape)

# Delete these two columns, as most values are 0
filtered_df.drop(['Work Pressure', 'Job Satisfaction'], axis=1, inplace=True)

# Delete the small number of rows where either of these two columns have 'Others'
filtered_df = filtered_df[~((filtered_df['Dietary Habits'] == 'Others') | (filtered_df['Sleep Duration'] == 'Others'))]
filtered_df = filtered_df[(filtered_df['Profession'] == 'Student')]
filtered_df.drop(['Profession'], axis=1, inplace=True)

# Delete the small number of rows where Financial Stress is '?', convert to float
filtered_df = filtered_df[~(filtered_df['Financial Stress'] == '?')]
filtered_df['Financial Stress'] = filtered_df['Financial Stress'].astype(float)

(27901, 17)


In [20]:
filtered_df.head()
#print(filtered_df.shape)
# print(filtered_df['Academic Pressure'].value_counts())
filtered_df['Academic Pressure'] = filtered_df['Academic Pressure'].replace(0.0, 1.0)

diet_map = {
    'Unhealthy': 0,
    'Moderate': 1,
    'Healthy': 2
}
filtered_df['Dietary Habits'] = filtered_df['Dietary Habits'].map(diet_map)

sleep_map = {
    "'Less than 5 hours'": 0,
    "'5-6 hours'": 1, 
    "'7-8 hours'": 2,
    "'More than 8 hours'": 3
}
filtered_df['Sleep Duration'] = filtered_df['Sleep Duration'].map(sleep_map)

if 'Gender' in filtered_df.columns:
    filtered_df['Sex'] = (filtered_df['Gender'] == 'Male').astype(int)
    filtered_df = filtered_df.drop(columns=['Gender'])

if 'Have you ever had suicidal thoughts ?' in filtered_df.columns:
    filtered_df['Suicidal_Thoughts'] = (filtered_df['Have you ever had suicidal thoughts ?'] == 'Yes').astype(int)
    filtered_df = filtered_df.drop(columns=['Have you ever had suicidal thoughts ?'])


if 'Family History of Mental Illness' in filtered_df.columns:
    filtered_df['Family_Mental_Illness'] = (filtered_df['Family History of Mental Illness'] == 'Yes').astype(int)
    filtered_df = filtered_df.drop(columns=['Family History of Mental Illness'])

In [21]:
#print((df['City'].value_counts()))

valid_cities = {
    'Kalyan', 'Srinagar', 'Hyderabad', 'Vasai-Virar', 'Lucknow', 'Thane', 
    'Ludhiana', 'Agra', 'Surat', 'Kolkata', 'Jaipur', 'Patna', 'Visakhapatnam', 
    'Pune', 'Ahmedabad', 'Bhopal', 'Chennai', 'Meerut', 'Rajkot', 'Delhi', 
    'Bangalore', 'Ghaziabad', 'Mumbai', 'Vadodara', 'Varanasi', 'Nagpur', 
    'Indore', 'Kanpur', 'Nashik', 'Faridabad'
}

city_corrections = {
    'Khaziabad': 'Ghaziabad',
}

#Replace obvious city mispellings
filtered_df['City'] = filtered_df['City'].replace(city_corrections)

#Create a mask for invalid cities, and replace them with Unknown
invalid_mask = ~filtered_df['City'].isin(valid_cities)
filtered_df.loc[invalid_mask, 'City'] = 'Unknown'

print((filtered_df['City'].value_counts()))

City
Kalyan           1563
Srinagar         1370
Hyderabad        1337
Vasai-Virar      1287
Lucknow          1154
Thane            1139
Ludhiana         1107
Agra             1090
Surat            1078
Kolkata          1064
Jaipur           1033
Patna            1006
Pune              968
Visakhapatnam     967
Ahmedabad         946
Bhopal            933
Chennai           884
Meerut            820
Rajkot            815
Bangalore         765
Delhi             765
Ghaziabad         744
Mumbai            697
Vadodara          692
Varanasi          684
Nagpur            649
Indore            642
Kanpur            606
Nashik            547
Faridabad         460
Unknown            25
Name: count, dtype: int64


In [22]:
print(filtered_df['Dietary Habits'].unique())
print(filtered_df['Sleep Duration'].unique())

[2 1 0]
[1 0 2 3]


In [23]:
filtered_df.head()

,Age,City,Academic Pressure,CGPA,Study Satisfaction,Sleep Duration,Dietary Habits,Degree,Work/Study Hours,Financial Stress,Depression,Sex,Suicidal_Thoughts,Family_Mental_Illness
id,,,,,,,,,,,,,,
2,33.0,Visakhapatnam,5.0,8.97,2.0,1,2,B.Pharm,3.0,1.0,1,1,1,0
8,24.0,Bangalore,2.0,5.90,5.0,1,1,BSc,3.0,2.0,0,0,0,1
26,31.0,Srinagar,3.0,7.03,5.0,0,2,BA,9.0,1.0,0,1,0,1
30,28.0,Varanasi,3.0,5.59,2.0,2,1,BCA,4.0,5.0,1,0,1,1
32,25.0,Jaipur,4.0,8.13,3.0,1,1,M.Tech,1.0,1.0,0,0,1,0


In [24]:
filtered_df.dtypes

Age                      float64
City                      object
Academic Pressure        float64
CGPA                     float64
Study Satisfaction       float64
Sleep Duration             int64
Dietary Habits             int64
Degree                    object
Work/Study Hours         float64
Financial Stress         float64
Depression                 int64
Sex                        int32
Suicidal_Thoughts          int32
Family_Mental_Illness      int32
dtype: object

In [25]:
filtered_df['Financial Stress'].value_counts()

Financial Stress
5.0    6698
4.0    5765
3.0    5213
1.0    5108
2.0    5053
Name: count, dtype: int64

In [26]:
# filtered_df.shape
# print(filtered_df['City'].unique())

In [27]:
filtered_df.to_csv("data/cleaned_student_depression_dataset.csv", index=False)